# Scaling to Multiple GPUs with Horovod

[Horovod](https://github.com/horovod/horovod) is a framework that is used for distributed deep learning training. It is compatible with TensorFlow, Keras, PyTorch, and Apache MXNet. In this lab, you will learn about Horovod and how to use it for distributing the training of the classification model across multiple GPUs.

## Lab Outline

The progression of this lab is as follows:

- We will work with a tensor-flow implementation for classifying images of the CIFAR 10. Get familiarised with the code, which you can access in the following link.
- We are going to refactor the code, adding the necessary steps for measuring performance
    - We will add code for measuring the throughput of the code in terms of images per second that are processed during training
    - As we do not know when the training will converge, we will refactor the code for stopping once a target-accuracy has been reached (we need this because we are going to experiment a lot)
    - We will refactor the code for measuring the total time employed by training
    - We will make the code parallel using Horovod



## Introduction to Horovod

Horovod was originally developed by Uber to make distributed deep learning fast and easy to use, bringing model training time down from days and weeks to hours and minutes. Today, the project is hosted by the [Linux Foundation](http://www.linuxfoundation.org) AI. 

With Horovod, an existing training script can be scaled up to run on hundreds of GPUs in just a few lines of Python code. Horovod can be installed on-premise or run out-of-the-box in cloud platforms, including [AWS](https://docs.aws.amazon.com/dlami/latest/devguide/activate-horovod.html), [Azure](https://github.com/Azure/BatchAI/tree/master/recipes/Horovod), and [Databricks](https://docs.databricks.com/applications/deep-learning/distributed-training/horovod-runner.html). Horovod can additionally run on top of Apache Spark, making it possible to unify data processing and model training into a single pipeline. Once Horovod has been configured, the same infrastructure can be used to train models with any framework, making it easy to switch between TensorFlow, PyTorch, MXNet, and future frameworks as machine learning tech stacks continue to evolve.





### Horovod's MPI Roots

Horovod, which is closely connected to MPI, follows the Single-Program Multiple-Data paradigm and requires attention to race conditions and process synchronization when distributing processes for model training.

Horovod assigns a unique numerical ID or rank to each process executing the program, which can be accessed programmatically to take steps such as identifying a process's rank in the code.

- To optimize the performance of Horovod, there are certain best practices that should be followed. These include:
- Pinning the process to its own exclusive GPU to avoid conflicts with other processes.
- Utilizing a single rank for broadcasting values that need to be used uniformly by all ranks to minimize communication overhead and improve performance.
- Utilizing a single rank for collecting and/or reducing values produced by all ranks to minimize communication overhead and improve performance.
- Utilizing a single rank for logging or writing to disk to avoid conflicts with other processes and improve performance.

By following these best practices, you can ensure that your Horovod training process runs smoothly and efficiently, with minimal communication overhead and conflicts with other processes.



## Refactoring the code

It is now time to begin refactoring our code as indicated above. Make sure not to move on to the next refactoring until the current one is complete. To verify that you are utilizing all of the GPUs in the system, run:

```bash
watch nvidia-smi
```




### Throughput

To print the throughput (images per second) in the code, modify it accordingly. Additionally, run the code for three epochs at the end of each epoch to obtain the required results.


In [ ]:

!python3 wideresnet.py --epochs 3




### Early Stopping

Modify the code to stop training once a target accuracy (target-accuracy) has been achieved for a consecutive number of epochs (patience). To evaluate the code, run the following command. The code should stop once the accuracy has reached 0.80 for three consecutive epochs. If this does not occur, the code should perform 100 epochs.


In [ ]:


!python3 wideresnet.py --epochs 100 --target-accuracy 0.80 --patience 3



### Printing time

Modify the code to print the total training time. Run the code to verify that it prints the time upon completion 


In [ ]:

!python3 wideresnet.py --epochs 100 --target-accuracy 0.80 --patience 3



### Modify the code to make it parallel with Horovod

The code needs to be modified to allow for training on multiple GPUs. Refer to the course slides for the necessary steps. If you need further documentation, consult the Horovod documentation at https://horovod.ai/.

After modifying the code, run the command `nvidia-smi` to check how many GPUs are available for the course. Save this number to a variable. For example, if your machine has 2 GPUs, run `export NUM_GPUS=2` in your terminal.

To test that your code works, run it using the following command:


In [ ]:

!horovodrun -np ${NUM_GPUS} python3 wideresnet.py --epochs 100 --target-accuracy  0.80 --patience 3



While the training runs, check with another terminal that all the GPUs on the system are in use during the training. 



### Verifying the code

If your parallel training throughput has increased but for performing the same number of epochs the wall time hasn’t reduce (or if the wall time has even increased), you probably need to check your code. There might be several reasons why this would make sense, like for example a the overhead due to communications (despite the graph above showed a relatively close to lineal performance for thousands of devices.) On a relatively small system, like the one where you probably have access to for this lab, that overhead should not be the responsible for equal or longer times and probably the issue is on your code. The most typical error in this case has to do with the number of steps performed by every GPU. 

In a parallel setup, the effective batch size used during training is the sum of the batches used in all the GPUs. If we do not indicate anything, when calling `[tf.keras.model.fit](http://tf.keras.model.fit)` the number of steps that the algorithm does per epoch is computed using the batch size and dataset size. The caveat is that that batch size is not the effective batch size mentioned above but the batch size within a single device (the advantage of this implementation is that slower devices could take smaller batch sizes than faster devices). If you do nothing, the effect of running the code would be the same as performing several epochs (instead of one!). Therefore, modify your code to be sure that per epoch the same number of input elements are considered as for the sequential training. 

Verify your code again by running 


In [ ]:

!horovodrun -np ${NUM_GPUS} python3 wideresnet.py --epochs 100  --target-accuracy  0.80 --patience 3



This exercise is only finished when you have achieved a higher throughput than the sequential case and for the same number of epochs, the training is faster. 




### Conclusion

You have now created a parallel version of the code. And this lab is finished. Do not throw away your modified program yet. There are some caveats that you might (or might have not) realised. In the next lab, we are going to modify our code for solving these, and we are going to compete with the rest of the class to see how can train the model faster within some given constraints.